In [14]:
import paramiko
from getpass import getpass

# Create the password prompt
thepass = getpass(prompt="Please enter your SSH password: ")

# Host information
host = "192.168.6.71"
port = 2222
username = "zachary.morin"
password = thepass

try:

    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(host, port, username, password)

except paramiko.AuthenticationException:
    print("Authentication failed")

In [ ]:
# Start SFTP session
sftp = ssh.open_sftp()

# This copies the kraken binary from the local directory to the server.
sftp.put('./kraken','/home/zachary.morin/kraken')

safe_processes = ["snapd"] # This is a whitelist of safe processes that are known to not be problematic.

# This command makes the kraken binary executable and then runs it with given paramets as a sudo user.
(stdin, stdout, stderr) = ssh.exec_command("sudo chmod +x /home/zachary.morin/kraken && sudo /home/zachary.morin/kraken --folder /usr/bin --folder  --folder /usr/sbin/   --folder /usr/local/bin  --folder /sbin  --folder /usr/local/sbin  --folder /bin", get_pty = True)
stdin.write(password + '\n')
z = stdout.readlines()[1:]

# This list gets the names of any sketchy programs that are not on the whitelist
sketchy_list = []
for x in range(1, len(z)):
    # I had to account for the special formatting that gives the words color
    if "[33mprocess[0m=" in z[x]:
        for words in safe_processes:
            if (("[33mprocess[0m="+words) in z[x]) == False:
                sketchy_list.append(z[x])

# This then gets the PIDs for each of the above gotten sketchy programs
sketchy_pids = []
for warns in sketchy_list:
    pid = warns.split(" ")
    pid = pid[8]
    pid = pid.split('=')
    pid = pid[1]
    sketchy_pids.append(pid)

# This then gets a list of lsof results for given PIDs that were obtained above 
lsof_results = []
for pids in sketchy_pids:
    (stdin, stdout, stderr) = ssh.exec_command("sudo lsof -p "+pids, get_pty = True)
    stdin.write(password + '\n')
    lsof_results.append(stdout.readlines()[2:])
print("lsof Command Results: \n")
for x in lsof_results:
    for y in x:
        print(y)

# This then gets any output from lsof that contains the word REG, indicating a normal file
REG_files = []
for result in lsof_results:
    list_test = result
    for x in range(0, len(list_test)):
        if "REG" in list_test[x]:
            REG_files.append(list_test[x])

# This then extracts the file locations from the above extrapolation, getting the file locations of REG files
file_locations = []
for reg in REG_files:
    test = reg.split("/")
    # To separate out and get the directories, I had to split using a forward slash and then rebuild the directories
    for x in range(1, len(test)):
        if x == 1:
            e = "/"+test[x]
        else:
            e = e + "/" + test[x]
    e = e[:-2]
    file_locations.append(e)
#file_locations.append("/var/log/auth.log") # To grab the SSH and user info

# Finally the files from those locations are downloaded to a local directory named "remote_files"
for eachFile in file_locations:

    # Create a list from the filename
    x = eachFile.split("/")

    # -1 is used to pluck the last element in a list.
    filename = x[-1]

    # Local path to store files
    local_path = "remote_files/" + filename
    sftp.get(eachFile, local_path)
    stdin.write(password + '\n')

# Closing the SSH connection
ssh.close()



What is the process name?

The process's name was "egrep" which is a well known tool.

Which user is running the process?

The user running the process is one "p.dalton".

Has the user logged in before via SSH?

The user has not logged in before via SSH, as the auth.log file has no recollection of any SSH attempts with that username. Even the older, archived logs had no information regarding the user.

When did the user login?

It is indeterminable from the auth logs.

How long was the login session?

It is indeterminable from the auth logs.

What is located in the unauthorized user’s home directory?

An executable file named "aqwerpuqwerj", "d.pid" (a normal file), and a directory named "test-lk".

Download any suspicious files.

Files are stored in the remote_files subdirectory.
